## Express Import API Call
This notebook illustrates an example to directly import an S3 asset and add it into a task in a project in one call. This example uses:

- `requests` library for API calls

In [ ]:
# Import required libraries
import requests

In [ ]:
# API URLs
auth_url = "https://api.centaurlabs.com/auth/public/v1/login"
tasks_url = "https://api.centaurlabs.com/tasks/public/v1/list?project_id="
imports_url = "https://api.centaurlabs.com/imports/public/v1/add"

In [ ]:
# Parameters for headers
email = "your_portal_email"  # Replace with your Centaur Email
api_key = "your_api_key_here"  # Replace with your Centaur API key
api_password = "your_api_password_here"  # Replace with your Centaur API password (not your portal password)

# Asset parameters
s3_uri = "s3://s3_uri" # Replace with your s3 URI
s3_region = "us-east-2" # Replace with your S3 Region

In [ ]:
# Function to get the authentication token
def get_auth_token(email, password, key):
    payload = {"username": email, "api_password": password}
    headers = {"X-API-KEY": key}
    response = requests.post(auth_url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("token")
    else:
        raise Exception(f"Authentication failed: {response.json()}")

In [ ]:
# Function to list tasks in the project. This can help you decide what tasks to us in the import function
def list_tasks(token, key, project_id):
    headers = {"Authorization": f"Bearer {token}", "X-API-KEY": key}
    url = f"{tasks_url}{project_id}"
    print(url)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['payload']
    else:
        raise Exception(f"Failed to list tasks: {response.json()}")

In [ ]:
# Function to run the express import into tasks
def add_import(key, token, project_id, tasks, uri, region):
    headers = {"Authorization": f"Bearer {token}", "X-API-KEY": key}
    payload = {
          "project_id": project_id,
          "source_type": "s3",
          "source_region": region,
          "uri": uri,
          "target_task_ids": tasks
    }
    response = requests.post(imports_url, headers=headers, json=payload)
    if response.status_code == 200:
        print("Import added successfully:", response.json())
    else:
        raise Exception(f"Failed to add import: {response.json()}")

In [ ]:
# Main script to run the process
try:
    # Step 1: Authenticate and get token
    token = get_auth_token(email, api_password, api_key)
    print(f"Authentication successful with token {token}")

    # Step 2: Get project ID from the user
    project_id = input("Enter the project ID: ")
        
    # Step 3: List all tasks in the project
    all_tasks = list_tasks(token, api_key, project_id)
    print(f"Found {len(all_tasks)} tasks in the project {project_id}")
        
    for task in all_tasks:
        print(f"Task ID: {task['task_id']}, Name: {task['task_name']}")

    # Step 4: input tasks to add
    task_input = input("Enter the tasks IDs you want to import to separated by commas: ")
    tasks = [
        int(task.strip()) for task in task_input.split(",") if task.strip()
    ]
        
    # Step 5: Add an import to the tasks
    if tasks:
        add_import(api_key, token, project_id, tasks, s3_uri, s3_region)
    else:
        add_import(api_key, token, project_id, s3_uri, s3_region)
    
except Exception as e:
    print("An error occurred:", str(e))